In [1]:
# Import required libraries and install any necessary packages
import spacy
import warnings

from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:
# reading the data in doccano jsonl output format
with open('data/admin.jsonl', 'r') as f:
    lines = list(f)

training_data: list = []

for line in lines:
    row = json.loads(line)
    if row['label']:
        training_data.append(  [ row["text"], { "entities": row["label"] } ] )

print(len(training_data))

149


In [3]:
training_data[0]

['"#1 fruit forward lots | 1 . 2 ell\'sworth black honey orange , berry fruit characteristics , green grapes & lemon crisp acidity . suitable for all brew methods . origin chikmagalur , karnataka process hand picked cherries are pulped , sun dried on raised beds and then eventually washed before they are hulled . dry aroma mild florals wet aroma red ripe fruits based on sensory evaluation orange , berry fruit characteristics , green grapes & lemon crisp acidity . med body , clean mouthfeel & long prevailing aftertaste . varietal chandragiri roast profile light-med roast with variable drum speed and a lower charge temperature to achieve a 13% dtr . altitude 1130 masl minimum resting period filter 6 days | espresso 14 days a song that pairs well roaster thoughts : as black honey produces fruit forward flavors and consists of jammy characteristics . we roasted this coffee at 205 degrees to extract the dense properties , fruitier flavor notes and a heavier body . after cupping the coffee a

In [4]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [5]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(training_data, test_size=0.1)

# Display the number of items in the training and testing sets
print(len(train), len(test))

# Open a file to log errors during annotation processing
file = open('train_file_try_2.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('train_doccano_try_2.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_doccano_try_2.spacy')

# Close the error log file
file.close()

134 15


/home/snehil/.dev_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 15/15 [00:00<00:00, 680.57it/s]


In [7]:
# configure spacy for custom NER model , using a base config file - https://spacy.io/usage/training#config

!python -m spacy init fill-config base_config.cfg config.cfg

/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Model Training

In [8]:
!python -m spacy train config.cfg  --output  output_try_2  --paths.train train_doccano_try_2.spacy  --paths.dev  test_doccano_try_2.spacy --gpu-id 0

/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Created output directory: output_try_2
ℹ Saving to output directory: output_try_2
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/ho

### Loading and trying the model with new data

In [24]:
# Import the spaCy library
import spacy

# Load the trained spaCy NER model from the specified path
nlp = spacy.load('output_try_2/model-best')


In [37]:
text = """
Jasmine kissed cranberry - 100 % arabica that is sourced from Chikmagalur, grown at an altitude of approximately 4,100ft to 4,500ft. It gets its unique name through a process which include a strain of yeast used to create a carbon dioxide-rich environment during fermentation in stainless-steel fermenters. A remarkable level of complexity in the beans is created by this process, laying the groundwork for a wonderful cup of coffee. Steady drying on raised beds is the next important step, which lets the flavors gradually develop and intensify. After another 30 days of continuous stirring, the coffee undergo additional drying developing scent of jasmine, complex notes of raspberries, cranberries, and sparkling malic acidity with a lingering floral aftertaste 
"""

In [38]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text.lower())

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

jasmine kissed cranberry -   ->>>>   NAME
arabica   ->>>>   COFFEE TYPE
chikmagalur   ->>>>   LOCATION
jasmine   ->>>>   TASTING NOTES
raspberries   ->>>>   TASTING NOTES
cranberries   ->>>>   TASTING NOTES
malic   ->>>>   ACIDITY


In [41]:
text ="""
Single origin Indian Coffee From Moganad Estate, Tamil Nadu which is a 100% Arabiaca coffee with 
an exquisite blend of balanced sweetness and brightness. A Medium Dark Roast coffee with 
flavor notes of Cocoa, Caramel and Nut which can also be enjoyed in a French press, moka pot, aeropress & espresso .
 It is washed processed, grown at an altitude of around 4430 ft

"""

In [42]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text.lower())

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

moganad   ->>>>   ESTATE
tamil nadu   ->>>>   LOCATION
medium dark   ->>>>   ROAST LEVEL
cocoa   ->>>>   TASTING NOTES
caramel   ->>>>   TASTING NOTES
nut   ->>>>   TASTING NOTES
4430 ft   ->>>>   ELEVATION
